In [80]:
import pandas as pd
import numpy as np

In [81]:
hague_cijfers = pd.read_excel('Hague_cijfers.xlsx',sheet_name = 'Meest-recent')

In [82]:
#buurts = hague_cijfers[hague_cijfers['Soort regio']=='Buurt']
buurt_data = hague_cijfers[['Naam van het gebied','Soort regio','Aantal inwoners','Mannen','Vrouwen','0 tot 15 jaar','15 tot 25 jaar','25 tot 45 jaar',
                 '45 tot 65 jaar','65 jaar of ouder','Gemiddeld inkomen per inkomensontvanger ','Actieven 15-75 jaar']]
buurt_data.columns = ['Name','type','Population','Male','Female','0 to 15 year','15 to 25 year','25 to 45 year',
                     '45 to 65 year','65 or older','Income','Activity rate']

In [83]:
buurt_data = buurt_data.reset_index(drop=True)
wijk = ''
buurt_data['Nearest wijk'] = 'Den Haag'
for i in range (0,len(buurt_data)):
    if (buurt_data.loc[i,'type'] == 'Wijk'):
        wijk = buurt_data.loc[i,'Name']
    buurt_data.loc[i,'Nearest wijk'] = wijk
buurt_data['Wijk no'] = buurt_data['Nearest wijk'].str.split(" ",n=2,expand=True)[1]

In [84]:
hague_primary_rate = '33'
hague_secondary_rate = '36'
hague_tertiary_rate = '31'
education = pd.read_csv('Education.csv')
education['Wijk no'] = education['Wijken'].str.split(" ",n = 1, expand = True)[0]


In [85]:
buurt_data_with_edu = buurt_data.merge(education,on=['Wijk no'])
buurt_data_with_edu.drop(['Nearest wijk', 'Wijken','Wijk no'], axis=1, inplace=True)
buurt_data_with_edu.rename(columns={'Opleidingsniveau Laag|2017': 'Primary', 'Opleidingsniveau Middelbaar|2017': 'Secondary','Opleidingsniveau Hoog|2017':'Tertiary'}, inplace=True)
buurt_data_final = buurt_data_with_edu
buurt_data_final['Primary'] = buurt_data_final['Primary'].fillna(hague_primary_rate)
buurt_data_final['Secondary'] = buurt_data_final['Secondary'].fillna(hague_secondary_rate)
buurt_data_final['Tertiary'] = buurt_data_final['Tertiary'].fillna(hague_tertiary_rate)
buurt_data_final['Primary'].replace({"x":hague_primary_rate}, inplace=True)
buurt_data_final['Secondary'].replace({"x":hague_secondary_rate}, inplace=True)
buurt_data_final['Tertiary'].replace({"x":hague_tertiary_rate}, inplace=True)
buurt_data_final['Primary'] = buurt_data_final['Primary'].astype('float')
buurt_data_final['Secondary'] = buurt_data_final['Secondary'].astype('float')
buurt_data_final['Tertiary'] = buurt_data_final['Tertiary'].astype('float')

In [86]:
buurt_data_final = buurt_data_final[buurt_data_final['type']=='Buurt']
buurt_data_final.drop(['type'],axis=1,inplace=True)

In [87]:
codes = pd.read_csv('buurt_codes.csv')
codes.rename(columns={'BUURTNAAM':'Name'},inplace=True)
codes = codes[['Name','BUURTCODE']]

In [88]:
buurt_data_final = buurt_data_final.merge(codes,on=['Name'],how='inner')
len(buurt_data_final)

114

In [89]:
buurt_data_final['Male'] = buurt_data_final['Male']/buurt_data_final['Population'] 
buurt_data_final['Female'] = buurt_data_final['Female']/buurt_data_final['Population']
buurt_data_final['0 to 15 year'] = buurt_data_final['0 to 15 year']/buurt_data_final['Population'] 
buurt_data_final['15 to 25 year'] = buurt_data_final['15 to 25 year']/buurt_data_final['Population'] 
buurt_data_final['25 to 45 year'] = buurt_data_final['25 to 45 year']/buurt_data_final['Population'] 
buurt_data_final['45 to 65 year'] = buurt_data_final['45 to 65 year']/buurt_data_final['Population'] 
buurt_data_final['65 or older'] = buurt_data_final['65 or older']/buurt_data_final['Population']
buurt_data_final['Primary'] = buurt_data_final['Primary']*0.01
buurt_data_final['Secondary'] = buurt_data_final['Secondary']*0.01
buurt_data_final['Tertiary'] = buurt_data_final['Tertiary']*0.01
buurt_data_final = buurt_data_final.dropna()
buurt_data_final = buurt_data_final[['Name','BUURTCODE','Population','Male','Female','0 to 15 year','15 to 25 year','25 to 45 year',
                                    '45 to 65 year','65 or older','Primary','Secondary','Tertiary','Activity rate','Income']] 

In [90]:
buurt_data_final.to_csv('buurt_data_input.csv',index=False)